# How can you seed with an LLM?

With a temperature of 0, the response of an AI agent will be the same every time. But what if you want a reproducable output that isn't deterministic?

Can a seed change that response meaningfully?

In [12]:
from creds import openai_key
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [13]:
prompt = PromptTemplate(
    input_variables=["seed"],
    template="""
    Give me one interesting baby name. Generate the name randomly with a seed of {seed}. 
    """,
)
llm = ChatOpenAI(temperature=0, openai_api_key=openai_key)
llm_chain = LLMChain(llm=llm, prompt=prompt)

With the same seed, we see no variance

In [14]:
seeds = [1]*10

for seed in seeds:
    print(llm_chain.run(seed=seed))

Zephyr
Zephyr
Zephyr
Zephyr
Zephyr
Zephyr
Zephyr
Zephyr
Zephyr
Zephyr


With numeric seeds, there is a tiny bit of variance

In [15]:
seeds = range(10)

for seed in seeds:
    print(llm_chain.run(seed=seed))

Avery
Zephyr
Zephyr
Aria
Zephyr
Aria
Aria
Aria
Aria
Aria


With larger more random numeric seeds, we don't see any benefit

In [16]:
import random

# Generate 10 large random integers
seeds = [random.randint(1000000000, 9999999999) for _ in range(10)]

for seed in seeds:
    print(llm_chain.run(seed=seed))

Zephyr
Aria
Aria
Zephyr
Zephyr
Zephyr
Zephyr
Zephyr
Zephyr
Zephyr


Using words instead for the seed means we need to rephrase the prompt

In [17]:
prompt = PromptTemplate(
    input_variables=["seed"],
    template="""
    Give me one interesting baby name. Be inspired by the word {seed}. 
    """,
)
llm = ChatOpenAI(temperature=0, openai_api_key=openai_key)
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [22]:
from faker import Faker

# Create a Faker object
faker = Faker()

# Generate 10 random words
seeds = [faker.word() for _ in range(10)]

# Display the random words
for word in seeds:
    print(word)

say
process
its
east
help
plant
occur
develop
different
bad


In [23]:
for seed in seeds:
    print(llm_chain.run(seed=seed))

Saylor
Procesia
Irisa - inspired by the word "iris", which is a beautiful flower with a unique shape and vibrant colors.
Easterly
Helperia
Sagebrush
Ocira (pronounced oh-SEER-ah)
Devlyn - a unique and modern name inspired by the word "develop" that means "one who is skilled in developing or growing something."
Dyfferentia (pronounced "dih-fuh-ren-shuh")
Badrinath - a unique and beautiful name inspired by the word "bad" which means "good" in Arabic. It is also the name of a famous Hindu pilgrimage site in India.


Those sure are some terrible names. A bit of prompt engineering can fix that.

In [24]:
prompt = PromptTemplate(
    input_variables=["seed"],
    template="""
    Give me one interesting baby name. Be very loosely inspired by the word {seed}. Don't explain yourself, just return the name.
    """,
)
llm = ChatOpenAI(temperature=0, openai_api_key=openai_key)
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [25]:
for seed in seeds:
    print(llm_chain.run(seed=seed))

Saylor
Prosecco
Istella
Ezra
Zelpha
Sage
Ocelyn
Devlin
Zephyr
Baxter


Kinda works. With some more prompt engineering, this word based seed approach could unlock reproducability